# ChatGPT Functions

This is ChatGPT response to "agents" and "tools". This notebook will be a playground for me to test how good it is.

In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import openai
import json
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt
from IPython.display import display, Markdown, Latex
import requests
import tiktoken
from tqdm import tqdm
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

## Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state

In [2]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
        # Auto call our function 
#         # HARD CODED
#         json_data["function_call"] = {"name": "get_well_arch_info"}

    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
def call_chat_api(messages, functions, model=GPT_MODEL):
    
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=messages,
        functions=functions,
        function_call= {"name": "get_well_arch_info"}
    ) 
    
    print(response)
    
    return response



In [4]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [5]:
# Initialize Vectorstore
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
vectorstore = FAISS.load_local("local_index", embeddings)
retriever = vectorstore.as_retriever()

In [6]:
# Create a function to get info from the book
def get_well_arch_info(query):
     
    docs = retriever.get_relevant_documents(query)
    doc_list = []
    for doc in docs:
        temp_json = {"content":doc.page_content,"source":doc.metadata['source']}
        doc_list.append(temp_json)
    
    return doc_list

In [7]:
# Test Function
query = "How to store data securely on Amazon S3"
get_well_arch_info(query)

[{'content': 'Amazon S3 supports several methods of encryption of your data at\n      rest. Using server-side encryption, Amazon S3 accepts your objects\n      as unencrypted data, and then encrypts them as they are stored.\n      Using client-side encryption, your workload application is\n      responsible for encrypting the data before it is sent to Amazon S3.\n      Both methods allow you to use AWS Key Management Service (AWS KMS)\n      to create and store the data key, or you can provide your own key,\n      which you are then responsible for. Using AWS KMS, you can set\n      policies using IAM on who can and cannot access your data keys and\n      decrypted data.',
  'source': 'https://docs.aws.amazon.com/wellarchitected/latest/framework/rel_backing_up_data_secured_backups_data.html'},
 {'content': 'Amazon S3 supports several methods of encryption of your data at\n      rest. Using server-side encryption, Amazon S3 accepts your objects\n      as unencrypted data, and then encry

In [8]:
# Init ChatGPT Functions
well_arch_functions = [
    
        {
        "name": "get_well_arch_info",
        "description": """Use this function to get guidance from the AWS well architected framework""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            User query. Responses should be use the context from the AWS well architected framework to help the user. Always provide the links to source material. 
                            """,
                }
            },
            "required": ["query"],
        }
        }
]

In [9]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]

    
#     response = call_chat_api(messages, functions)
#     full_message = response["choices"][0]
#     print("Auto call function")
#     return call_well_arch_function(messages, full_message)

    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_well_arch_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()
    
def call_well_arch_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary."""

    if full_message["message"]["function_call"]["name"] == "get_well_arch_info":
        try:
            parsed_output = json.loads(
                full_message["message"]["function_call"]["arguments"]
            )
            print("Getting info from AWS well architected framework")
            print(f"User query: {parsed_output['query']}")
            guidance = get_well_arch_info(parsed_output["query"])
        except Exception as e:
            print(parsed_output)
            print(f"Function execution failed")
            print(f"Error message: {e}")
        
        # Added the function message
        messages.append(
            {
                "role": "function",
                "name": full_message["message"]["function_call"]["name"],
                "content": str(guidance),
            }
        )
    
        # Send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model=GPT_MODEL,
            messages=messages
        )
        return second_response    

In [10]:
# Start with a system message
system_message = """You are an expert AWS Certified Solutions Architect. Your role is to help customers understand best practices on building on AWS. Return your response in markdown, so you can bold and highlight important steps for customers. If the answer cannot be found within the context, write 'I could not find an answer'""" 
convo = Conversation()
convo.add_message("system", system_message)

In [11]:
# Add a user message
convo.add_message("user", "Hi, How to store data securely on Amazon S3? use info from the AWS well architected framework")
chat_response = chat_completion_with_function_execution(
    convo.conversation_history, functions=well_arch_functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
convo.add_message("assistant", assistant_message)
display(Markdown(assistant_message))


Function generation requested, calling function
Getting info from AWS well architected framework
User query: store data securely on Amazon S3


To store data securely on Amazon S3, you can follow these best practices outlined in the AWS Well-Architected Framework:

1. **Access Control**: Apply **AWS Identity and Access Management (IAM)** policies to grant least privilege access to S3 buckets. Avoid using AWS root credentials and enable multi-factor authentication (MFA) for account access.

2. **Encryption**: Use **Server-Side Encryption (SSE)** to protect data at rest within S3. SSE provides options like S3 Managed Keys (SSE-S3), AWS Key Management Service (SSE-KMS), or Server-Side Encryption with AWS Key Management Service (SSE-C).

3. **Access Logging**: Enable **S3 server access logging** to record all requests made to your bucket and analyze potential security breaches or unauthorized access attempts.

4. **Bucket Policies**: Use **bucket policies** to control access to S3 buckets from specific IP ranges or specific VPC endpoints. Configure the policies to deny public access by default and allow only necessary access privileges.

5. **Lifecycle Management**: Implement **S3 lifecycle policies** to automatically transition objects to lower-cost storage classes like Amazon S3 Glacier or S3 Glacier Deep Archive after a specified time. This helps optimize costs while ensuring data availability.

6. **Versioning**: Enable **S3 versioning** for your buckets to maintain multiple versions of an object. This helps in recovering from accidental modifications or deletions.

7. **Cross-Region Replication**: Implement **cross-region replication** (CRR) for critical data to replicate objects across different AWS regions for disaster recovery and high availability. Use CRR with SSE-KMS encryption for enhanced security.

8. **Data Backup**: Consider backing up critical data using **AWS Storage Gateway** or **AWS DataSync**. These services help backup on-premises data to the AWS Cloud using S3 buckets. You can also leverage different storage tiers like Amazon S3 Glacier or S3 Glacier Deep Archive to reduce storage costs.

Remember to regularly review and audit your S3 settings and configurations to ensure ongoing security and compliance with your organization's policies.

For more detailed information, please refer to the following AWS Well-Architected Framework documentation sources:

- [AWS Well-Architected Framework - Security Pillar](https://docs.aws.amazon.com/wellarchitected/latest/framework/sus_sus_data_a7.html)
- [AWS Well-Architected Framework - Performance Efficiency Pillar - Storage](https://docs.aws.amazon.com/wellarchitected/latest/framework/perf-storage.html)
- [AWS Well-Architected Framework - Reliability Pillar - Backing Up Data](https://docs.aws.amazon.com/wellarchitected/latest/framework/rel_backing_up_data_identified_backups_data.html)

In [12]:
# Add another user message to induce our system to use the second tool
convo.add_message(
    "user",
    "Thats great, how about for EC2?",
)
updated_response = chat_completion_with_function_execution(
    convo.conversation_history, functions=well_arch_functions
)
display(Markdown(updated_response["choices"][0]["message"]["content"]))

Function generation requested, calling function
Getting info from AWS well architected framework
User query: How to secure EC2 instances on AWS?


To secure your EC2 instances on AWS, you can follow these best practices from the AWS Well-Architected Framework:

1. **Perform Vulnerability Management**: Regularly scan your EC2 instances for vulnerabilities using automated tools such as **Amazon Inspector** or third-party solutions. This helps identify and remediate any security weaknesses.

2. **Reduce Attack Surface**: Minimize the attack surface by reducing unused components and services. Remove unnecessary operating system packages, applications, and external software modules in your code. Refer to hardening and security configuration guides, such as those provided by the **Center for Internet Security (CIS)**, to ensure secure configurations.

3. **Implement Managed Services**: Utilize managed services provided by AWS, such as **AWS Identity and Access Management (IAM)** and **AWS Security Groups**. IAM allows you to manage user access and permissions, while Security Groups act as virtual firewalls to control inbound and outbound traffic to your instances.

4. **Automate Compute Protection**: Automate security tasks using **AWS CloudFormation** or **AWS Systems Manager Automation**. This ensures consistency, efficiency, and reduces the risk of manual errors.

5. **Enable People to Perform Actions at a Distance**: Grant access to EC2 instances remotely using secure methods like **AWS Systems Manager Session Manager** instead of relying on direct SSH or RDP access. This enhances security by limiting exposure to remote access vulnerabilities.

6. **Validate Software Integrity**: Establish a process to verify the integrity of software running on your EC2 instances. Use tools like **AWS CodePipeline** and **AWS CodeCommit** to ensure only authorized and approved code changes are deployed.

Regularly reviewing and updating your security controls, patching vulnerabilities, and monitoring logs and alerts are crucial steps to maintaining the security of your EC2 instances.

For more detailed information, refer to the following AWS Well-Architected Framework documentation sources:

- [AWS Well-Architected Framework - Security Pillar](https://docs.aws.amazon.com/wellarchitected/latest/framework/sec-06.html)
- [AWS Well-Architected Framework - Security Pillar - Protect Compute](https://docs.aws.amazon.com/wellarchitected/latest/security-pillar/sec_protect_compute_auto_protection.html)
- [AWS Well-Architected Framework - Security Pillar - Reduce Attack Surface](https://docs.aws.amazon.com/wellarchitected/latest/framework/sec_protect_compute_reduce_surface.html)